WebScrap no site indeed.com para análise das vagas.

##Primeira coisa: Instalar pip install bs4 e pip install requests

In [1]:
import csv
import bs4
from datetime import datetime
import requests
from bs4 import BeautifulSoup


Selecionando/Gerando a URL pelos parametros de campo de trabalho e localização


In [10]:
def get_url(campo, localizacao):
    template = 'https://br.indeed.com/jobs?q={}&l={}'
    campo = campo.replace(' ', '+')
    localizacao = localizacao.replace(' ', '+')
    url = template.format(campo, localizacao)
    return url

In [11]:
url = get_url('estagio', 'sao paulo')
print(url)

https://br.indeed.com/jobs?q=estagio&l=sao+paulo


Proximo passo é mandar uma requisição GET e extrair o html bruto

In [12]:
response = requests.get(url)


In [13]:
response
##Verificando se a requisição foi feita com sucesso. Se voltar 200 É OK!!!

<Response [200]>

In [45]:
soup = BeautifulSoup(response.text, 'html.parser')
cards = soup.find_all('div', 'job_seen_beacon')
len(cards)



15

Prototipando o modelo com registros únicos para validar cada tag html. Depois será realizado uma função com todos os modelos juntos.

In [72]:
card = cards[0]
job_title = card.h2.a.span.get('title')
print(job_title)
company = card.find('span', 'companyName').text.strip()
print(company)
job_location = card.find('div', 'companyLocation').text.strip()
print(job_location)


## NAS PRÓXIMAS VERSÕES ADICIONAR: TEMPO DE POST, DIA DA EXTRAÇÃO, DESCRIÇÃO E SALÁRIO.

Jovem Aprendiz | Administrativo [Eficiência Operacional]
Fisia - Corporativo
São Paulo, SP


Com as tags validadas, o proximo passo é criar uma função com o modelo geral com os registros.

In [86]:
def get_record(card):
    job_title = card.h2.a.span.get('title')
    company = card.find('span', 'companyName').text.strip()
    job_location = card.find('div', 'companyLocation').text.strip()
    
        
    record = (job_title, company, job_location)
    return record

Adicionando as vagas em uma lista


In [74]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)

Criando um laço para não parar na primeira página de vagas e ir até a ultima

In [75]:
while True:
    try:
        url = 'https://www.br.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
    except AttributeError:
        break

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'job_seen_beacon')

    for card in cards:
        record = get_record(card)
        records.append(record)
        

Criando um escopo com todos os código + exportação pro excel (Não sei extrair para o banco de dados)

In [89]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup


def get_url(campo, localizacao):
    template = 'https://br.indeed.com/jobs?q={}&l={}'
    campo = campo.replace(' ', '+')
    localizacao = localizacao.replace(' ', '+')
    url = template.format(campo, localizacao)
    return url

def get_record(card):
    job_title = card.h2.a.span.get('title')
    company = card.find('span', 'companyName').text.strip()
    job_location = card.find('div', 'companyLocation').text.strip()
    
        
    record = (job_title, company, job_location)
    return record


def main(position, location):
    """Run the main program routine"""
    records = []
    url = get_url(position, location)
    
    # extract the job data
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'job_seen_beacon')
        for card in cards:
            record = get_record(card)
            records.append(record)
        try:
            url = 'https://www.br.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
        except AttributeError:
            break
        
    # save the job data
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle', 'Company', 'Location'])
        writer.writerows(records)


In [90]:
main('Java','Sao Paulo')

AttributeError: 'NoneType' object has no attribute 'span'